**NOTEBOOK NÀY ĐÃ CONFIG CÁC PARAMETTERS ĐỂ MÔ HÌNH CHO KẾT QUẢ TỐT NHẤT**

## IMPORT LIBRARY

In [ ]:
!pip install pyspark
!pip install recommenders
!pip install surprise
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession, Row
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
import numpy as np
import pandas as pd
import zipfile
import math
from recommenders.evaluation.python_evaluation import ndcg_at_k
from surprise.model_selection.split import LeaveOneOut

     |████████████████████████████████| 281.4 MB 15 kB/s 
     |████████████████████████████████| 198 kB 45.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=a1df1a546d75f4014389c15f2dc11958f43c2be99ebc24ec06ec56502a1f28e3
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
     |████████████████████████████████| 335 kB 19.3 MB/s 
     |████████████████████████████████| 61 kB 6.4 MB/s 
     |████████████████████████████████| 197 kB 39.4 MB/s 
     |████████████████████████████████| 4.0 MB 51.8 MB/s 
     |████████████████████████████████| 11.8 MB 38.1 MB/s 
     |████████████████████████████████| 1.5 MB 38.1 MB/s 
     |████████████████████████████████| 12.4 MB 43.9 MB/s 
     |████████████████████████████████| 310 kB 46.1 MB/s 
     |████████████████████████████████| 636 kB 37.0 MB/s 
     |████████████████████████████████| 749 kB 51.2 MB/s 
     |████

## MOVIELENS 100K, 1M, 10M RECORD.



In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!wget https://files.grouplens.org/datasets/movielens/ml-10m.zip
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip

--2022-04-12 10:59:21--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  26.9MB/s    in 0.2s    

2022-04-12 10:59:22 (26.9 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

--2022-04-12 10:59:22--  https://files.grouplens.org/datasets/movielens/ml-10m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65566137 (63M) [application/zip]
Saving to: ‘ml-10m.zip’

ml-10m.zip          100%[===================>]  62.53M  77.5MB/s    in 0.8s    

2022-04-12 10:59:23 (77.5 MB/s) - ‘ml-10m.zip’ saved [655

In [ ]:
with zipfile.ZipFile("/content/ml-1m.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/MOVIE")
with zipfile.ZipFile("/content/ml-10m.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/MOVIE")
with zipfile.ZipFile("/content/ml-100k.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/MOVIE")

## MovieLens 1M ratings




### Ratings

In [ ]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
ratings_data=sc.textFile("/content/MOVIE/ml-1m/ratings.dat")

In [ ]:
ratings_data.take(1)

['1::1193::5::978300760']

In [ ]:
ratings=ratings_data.map(lambda line: line.split("::")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()

In [ ]:
ratings.take(1)

[('1', '1193', '5')]

### Movies

In [ ]:
movies_data=sc.textFile("/content/MOVIE/ml-1m/movies.dat")

In [ ]:
movies_data.take(1)

["1::Toy Story (1995)::Animation|Children's|Comedy"]

In [ ]:
movies=movies_data.map(lambda line: line.split("::")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()

In [ ]:
movies.take(1)

[('1', 'Toy Story (1995)', "Animation|Children's|Comedy")]

### Users

In [ ]:
users_data=sc.textFile("/content/MOVIE/ml-1m/users.dat")

In [ ]:
users_data.take(1)

['1::F::1::10::48067']

In [ ]:
users=users_data.map(lambda line: line.split("::")).map(lambda tokens: (tokens[0],tokens[1],tokens[2],tokens[3],tokens[4])).cache()

In [ ]:
users.take(1)

[('1', 'F', '1', '10', '48067')]

### RECOMMENDER SYSTEM

In [ ]:
train, test = ratings.randomSplit([7, 3], seed=0)
train_for_prdict=train.map(lambda x: ((x[0],x[1]),x[2]))
test_for_predict=test.map(lambda x: (x[0], x[1]))

In [ ]:
train.take(1)

[('1', '1193', '5')]

In [ ]:
test_for_predict.take(1)

[('1', '914')]

Ranks: số lượng feature sẽ sử dụng.  
lambda:  tham số chính quy hóa trong ALS.  
interations: số lần lặp lại ALS sẽ chạy. Thường bé hơn hoặc bằng 20.

#### COMPLETED MODEL

##### RMSE

In [ ]:
seed = 1
model = ALS.train(train, rank=7, seed=1, iterations=20,lambda_=0.05)
predictions = model.predictAll(test_for_predict)
rates_and_preds = test.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions.map(lambda r: ((r[0], r[1]), r[2])))
RMSE_error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
print('(iterations 20, rank 7, lambda 0.05): The RMSE is %s' % (RMSE_error))




(iterations 20, rank 7, lambda 0.05): The RMSE is 0.8624043509387181


##### NDCG 

In [ ]:
test_df=pd.DataFrame(data=test.collect(),columns=["userID","itemID","rating"])
test_df['rating'] = test_df['rating'].astype(float, errors = 'raise')
test_df['userID'] = test_df['userID'].astype(int, errors = 'raise')
test_df['itemID'] = test_df['itemID'].astype(int, errors = 'raise')
model = ALS.train(train, rank=10, seed=1, iterations=20,lambda_=0.15)
predictions = model.predictAll(test_for_predict)
rates_and_preds = test.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions.map(lambda r: ((r[0], r[1]), r[2])))
predict_df=pd.DataFrame(data=predictions.collect(),columns=["userID","itemID","prediction"])
predict_df['prediction'] = predict_df['prediction'].astype(float, errors = 'raise')
predict_df['userID'] = predict_df['userID'].astype(int, errors = 'raise')
predict_df['itemID'] = predict_df['itemID'].astype(int, errors = 'raise')

NDCG_error=ndcg_at_k(test_df,predict_df)
print('(iterations 20, rank 10, lambda 0.15): The NDCG is %s' % (NDCG_error))

(iterations 20, rank 10, lambda 0.15): The NDCG is 0.9999259423752931


##### HIT RATE

#### REVIEWS

Tuỳ theo từng độ đo mà interations rank và lambda được gán với giá trị khác nhau

### Sparsity data [MovieLens 100K ratings]


Tương tự Mean Square Erorr thì ta sử dụng MAE(Mean Absolute Error) để làm tiêu chuẩn độ đo cho câu hỏi này.


u.data ở đây chứa 100000 ratings bởi 943 users với 1682 items gồm 4 cột:  
user id | item id | rating | timestamp.  
Chúng ta sẽ lấy 3 cột đầu và bỏ cột cuối cùng.

In [ ]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
ratings_data=sc.textFile("/content/MOVIE/ml-100k/u.data")
ratings=ratings_data.map(lambda line: line.split("\t")).map(lambda tokens: (int(tokens[0]),int(tokens[1]),int(tokens[2]))).cache()
train, test = ratings.randomSplit([7, 3], seed=0)
train_for_prdict=train.map(lambda x: ((x[0],x[1]),x[2]))
test_for_predict=test.map(lambda x: (x[0], x[1]))

In [ ]:

seed = 1
model = ALS.train(train, rank=3, seed=1, iterations=20,lambda_=0.05)
predictions = model.predictAll(test_for_predict)
rates_and_preds = test.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions.map(lambda r: ((r[0], r[1]), r[2])))

MAE_error = rates_and_preds.map(lambda r: np.abs(r[1][0] - r[1][1])).sum()/len(rates_and_preds.collect())
print('(iterations 20, rank 3, lambda 0.05): The best MAE is %s' % (MAE_error))

(iterations 20, rank 3, lambda 0.05): The best MAE is 0.7354813932904432


### Bigger data [MovieLens 10M ratings]

#### Ratings

Dựa trên những dữ liệu trước ta thấy hầu như iterations = 20 là tốt nhất. Nên ở đây chúng ta chỉ cần tìm lambda và rank để tránh mất quá nhiều thời gian vì dữ liệu lớn hơn trước gấp 10 lần. Thời gian train sẽ tăng lên ít nhất 10*k lần so với 1M ratings ( Hiện tại để train 7M ratings thì cần hơn 6 tiếng nhưng vẫn chưa hoàn thành).

**Do đó ta quyết định rời rạc hoá dữ liệu: Tức là giảm bớt số lượng ratings đi ít nhất 30% mà vẫn đảm bảo số lượng người đã ratings không thay đổi**

In [ ]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
ratings_data=sc.textFile("/content/MOVIE/ml-10M100K/ratings.dat")
ratings=ratings_data.map(lambda line: line.split("::")).map(lambda tokens: (int(tokens[0]),int(tokens[1]),float(tokens[2])))

In [ ]:
df = pd.DataFrame(data=ratings.collect(),columns=["userId","movieId","ratings"])
np.random.seed(1)
dictUser=dict(df.userId.value_counts())
count=0
listIndex=[]
for index in np.random.choice(range(0,10000000),size=8000000,replace=False):
      User_id=df.loc[int(index)].userId
      if(dictUser[User_id]>30):
        dictUser[User_id]=dictUser[User_id] - 1
        count = count + 1
        listIndex.append(index)

In [ ]:
df=df.drop(listIndex)

In [ ]:
df

,userId,movieId,ratings
0,1,122,5.0
1,1,185,5.0
2,1,231,5.0
3,1,292,5.0
4,1,316,5.0
...,...,...,...
10000049,71567,2107,1.0
10000050,71567,2126,2.0
10000051,71567,2294,5.0
10000052,71567,2338,2.0


In [ ]:
df[df.userId<26000]

,userId,movieId,ratings
0,1,122,5.0
1,1,185,5.0
2,1,231,5.0
3,1,292,5.0
4,1,316,5.0
...,...,...,...
3616127,25999,1304,4.0
3616130,25999,1370,4.0
3616131,25999,1371,3.0
3616132,25999,1372,3.0


In [ ]:
df[df.userId<26000].to_csv("10M_Ratings_After_Preprocess.dat",sep = "|",header=False)

In [ ]:
ratings_data=sc.textFile("/content/10M_Ratings_After_Preprocess.dat")
ratings=ratings_data.map(lambda line: line.split("|")).map(lambda tokens: (int(tokens[1]),int(tokens[2]),float(tokens[3])))

In [ ]:
train, test = ratings.randomSplit([7, 3], seed=0)
train_for_prdict=train.map(lambda x: ((x[0],x[1]),x[2]))
test_for_predict=test.map(lambda x: (x[0], x[1]))
seed = 1
iterations = [20]
regularization_parameters = [0.1]
ranks = [3]
MAE_err=[]
for iteration in iterations:
  for rank in ranks:
    for regularization_parameter in regularization_parameters:
      model = ALS.train(train, rank=rank, seed=seed, iterations=iteration,lambda_=regularization_parameter)
      predictions = model.predictAll(test_for_predict)
      rates_and_preds = test.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions.map(lambda r: ((r[0], r[1]), r[2])))

      MAE_error = rates_and_preds.map(lambda r: np.abs(r[1][0] - r[1][1])).sum()/len(rates_and_preds.collect())
      MAE_err.append(MAE_error)
      print('(iterations %s, rank %s, lambda %s): The MAE is %s' % (iteration,rank,regularization_parameter,MAE_error))
